# Creating Dataframe

In [2]:
import requests
import pandas as pd

In [2]:
url_start="https://statsapi.web.nhl.com/api/v1/game/{}/feed/live"

In [3]:
game_id=[2022020022, 2022020028, 2022020044, 2022020072, 2022020088, 2022020102, 
         2022020111, 2022020132, 2022020144, 2022020164, 2022020181, 2022020190, 
         2022020203, 2022020217, 2022020237, 2022020255, 2022020272, 2022020279, 
         2022020311, 2022020325, 2022020348, 2022020361, 2022020386, 2022020394, 
         2022020405, 2022020419, 2022020435, 2022020460, 2022020467, 2022020485, 
         2022020503, 2022020519, 2022020560, 2022020563, 2022020584]

In [4]:
## Saving data from each ID 
for id in game_id:
    url=url_start.format(id)
    data=requests.get(url)
    
    with open("Individual Game Data/Game ID{}.html".format(id), "w+") as f:
        f.write(data.text)

In [5]:
x=[]
y=[]
event=[]
name=[]
vs=[]
strength=[]
period=[]
pt=[]
ptype=[]
en=[]
rinkside=[]
stype=[]
dt=[]
gameid=[]

In [7]:
## Appending all data from games 
for id in game_id:
    with open("Individual Game Data/Game ID{}.html".format(id)) as f:
        page=f.read()
        
    import json
    response_info=json.loads(page)
    
    away_team=response_info['gameData']['teams']['away']['name']
    home_team=response_info['gameData']['teams']['home']['name']

    for i in response_info['liveData']['plays']['allPlays']: 
        if i['result']['event']=='Shot' or i['result']['event']=='Goal'or i['result']['event']=='Blocked Shot'or i['result']['event']=='Missed Shot':
            x.append(i['coordinates']['x'])
            y.append(i['coordinates']['y'])
            event.append(i['result']['event'])
            name.append(i['team']['name'])
            team_name=(i['team']['name'])
            period.append(i['about']['period'])
            period_num=(i['about']['period'])
            pt.append(i['about']['periodTime'])
            ptype.append(i['about']['periodType'])
            try:
                stype.append(i['result']['secondaryType'])
            except: 
                stype.append('NA')
            try: 
                strength.append(i['result']['strength']['code'])
            except:
                strength.append('NA')
            try:
                en.append(i['result']['emptyNet'])
            except: 
                en.append('NA')
            dt.append(response_info['gameData']['datetime']['dateTime'])
            gameid.append(response_info['gameData']['game']['pk'])
            
            if team_name==away_team:
                vs.append('Away')
                team='away'
            else: 
                vs.append('Home')
                team='home'
                
            for b in response_info['liveData']['linescore']['periods']:
                if b['num']== period_num and team=='away':
                    rinkside.append(b['away']['rinkSide'])
                elif b['num']== period_num and team=='home':
                    rinkside.append(b['home']['rinkSide'])
            if period_num==5:
                rinkside.append('SO')
        

In [8]:
data1 = {'X': x,
        'Y': y,
        'Event': event,
        'Shot_Type': stype,
        'Team_Name': name,
        'Team_Type':vs,
        'Strength': strength,
        'Period': period,
        'Period_Time':pt,
        'Period_Type':ptype,
        'Rink_Side': rinkside,
        'Empty_Net':en,
        'Date_Time':dt,
        'Game_Id': gameid
        }

In [9]:
df=pd.DataFrame(data1)
df

,X,Y,Event,Shot_Type,Team_Name,Team_Type,Strength,Period,Period_Time,Period_Type,Rink_Side,Empty_Net,Date_Time,Game_Id
0,46.0,-24.0,Shot,Slap Shot,Montréal Canadiens,Away,NA,1,00:33,REGULAR,left,NA,2022-10-14T23:00:00Z,2022020022
1,65.0,41.0,Shot,Wrist Shot,Montréal Canadiens,Away,NA,1,00:42,REGULAR,left,NA,2022-10-14T23:00:00Z,2022020022
2,70.0,-19.0,Shot,Wrist Shot,Montréal Canadiens,Away,NA,1,00:53,REGULAR,left,NA,2022-10-14T23:00:00Z,2022020022
3,-58.0,11.0,Shot,Snap Shot,Detroit Red Wings,Home,NA,1,01:07,REGULAR,right,NA,2022-10-14T23:00:00Z,2022020022
4,3.0,-5.0,Shot,Wrist Shot,Montréal Canadiens,Away,NA,1,01:23,REGULAR,left,NA,2022-10-14T23:00:00Z,2022020022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,-74.0,22.0,Shot,Snap Shot,Detroit Red Wings,Home,NA,3,17:37,REGULAR,right,NA,2023-01-01T00:00:00Z,2022020584
3981,56.0,9.0,Shot,Wrist Shot,Ottawa Senators,Away,NA,3,18:42,REGULAR,left,NA,2023-01-01T00:00:00Z,2022020584
3982,77.0,14.0,Blocked Shot,NA,Detroit Red Wings,Home,NA,3,18:45,REGULAR,right,NA,2023-01-01T00:00:00Z,2022020584
3983,68.0,-29.0,Shot,Wrist Shot,Ottawa Senators,Away,NA,3,19:48,REGULAR,left,NA,2023-01-01T00:00:00Z,2022020584


In [10]:
## Saving dataframe 
df.to_csv('Shots.csv')

# Cleaning Data 

In [3]:
## Read saved data to dataframe
sdf=pd.read_csv('Shots.csv')
## Delete row number column
del sdf['Unnamed: 0']

In [4]:
## Checking for duplicated rows 
duplicate = sdf[sdf.duplicated()]
duplicate.sum()

X              0.0
Y              0.0
Event          0.0
Shot_Type      0.0
Team_Name      0.0
Team_Type      0.0
Strength       0.0
Period         0.0
Period_Time    0.0
Period_Type    0.0
Rink_Side      0.0
Empty_Net      0.0
Date_Time      0.0
Game_Id        0.0
dtype: float64

In [5]:
## Checking for missing values 
sdf.isnull().sum()
## All columns with missing values dont always have values 

X                 0
Y                 0
Event             0
Shot_Type      1828
Team_Name         0
Team_Type         0
Strength       3756
Period            0
Period_Time       0
Period_Type       0
Rink_Side         0
Empty_Net      3761
Date_Time         0
Game_Id           0
dtype: int64

In [6]:
sdf.dtypes

X              float64
Y              float64
Event           object
Shot_Type       object
Team_Name       object
Team_Type       object
Strength        object
Period           int64
Period_Time     object
Period_Type     object
Rink_Side       object
Empty_Net       object
Date_Time       object
Game_Id          int64
dtype: object

In [9]:
## Converting Datetime UTC to Date EST
sdf['DT_Standard'] = pd.to_datetime(sdf['Date_Time'])
sdf['Date']=sdf['DT_Standard'].dt.tz_convert('US/Central')
sdf['Date'] = pd.to_datetime(sdf['Date']).dt.date
sdf

,X,Y,Event,Shot_Type,Team_Name,Team_Type,Strength,Period,Period_Time,Period_Type,Rink_Side,Empty_Net,Date_Time,Game_Id,DT_Standard,Date
0,46.0,-24.0,Shot,Slap Shot,Montréal Canadiens,Away,NaN,1,00:33,REGULAR,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
1,65.0,41.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,00:42,REGULAR,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
2,70.0,-19.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,00:53,REGULAR,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
3,-58.0,11.0,Shot,Snap Shot,Detroit Red Wings,Home,NaN,1,01:07,REGULAR,right,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
4,3.0,-5.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,01:23,REGULAR,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,-74.0,22.0,Shot,Snap Shot,Detroit Red Wings,Home,NaN,3,17:37,REGULAR,right,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3981,56.0,9.0,Shot,Wrist Shot,Ottawa Senators,Away,NaN,3,18:42,REGULAR,left,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3982,77.0,14.0,Blocked Shot,NaN,Detroit Red Wings,Home,NaN,3,18:45,REGULAR,right,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3983,68.0,-29.0,Shot,Wrist Shot,Ottawa Senators,Away,NaN,3,19:48,REGULAR,left,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31


In [10]:
## Period Type changed to proper case 
sdf['Period_Type'] = sdf['Period_Type'].str.title()
sdf.head()

,X,Y,Event,Shot_Type,Team_Name,Team_Type,Strength,Period,Period_Time,Period_Type,Rink_Side,Empty_Net,Date_Time,Game_Id,DT_Standard,Date
0,46.0,-24.0,Shot,Slap Shot,Montréal Canadiens,Away,NaN,1,00:33,Regular,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
1,65.0,41.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,00:42,Regular,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
2,70.0,-19.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,00:53,Regular,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
3,-58.0,11.0,Shot,Snap Shot,Detroit Red Wings,Home,NaN,1,01:07,Regular,right,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
4,3.0,-5.0,Shot,Wrist Shot,Montréal Canadiens,Away,NaN,1,01:23,Regular,left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14


In [11]:
## Changing all shot types to same format
sdf['Shot_Type'] = sdf['Shot_Type'].str.replace('-',' ')

## Team name modifying acute accent 
sdf['Team_Name'] = sdf['Team_Name'].str.replace('é','e')

## Proper case for Rink_side (right and left)
sdf['Rink_Side'] = sdf['Rink_Side'].str.replace('left','Left')
sdf['Rink_Side'] = sdf['Rink_Side'].str.replace('right','Right')

In [12]:
sdf

,X,Y,Event,Shot_Type,Team_Name,Team_Type,Strength,Period,Period_Time,Period_Type,Rink_Side,Empty_Net,Date_Time,Game_Id,DT_Standard,Date
0,46.0,-24.0,Shot,Slap Shot,Montreal Canadiens,Away,NaN,1,00:33,Regular,Left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
1,65.0,41.0,Shot,Wrist Shot,Montreal Canadiens,Away,NaN,1,00:42,Regular,Left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
2,70.0,-19.0,Shot,Wrist Shot,Montreal Canadiens,Away,NaN,1,00:53,Regular,Left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
3,-58.0,11.0,Shot,Snap Shot,Detroit Red Wings,Home,NaN,1,01:07,Regular,Right,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
4,3.0,-5.0,Shot,Wrist Shot,Montreal Canadiens,Away,NaN,1,01:23,Regular,Left,NaN,2022-10-14T23:00:00Z,2022020022,2022-10-14 23:00:00+00:00,2022-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,-74.0,22.0,Shot,Snap Shot,Detroit Red Wings,Home,NaN,3,17:37,Regular,Right,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3981,56.0,9.0,Shot,Wrist Shot,Ottawa Senators,Away,NaN,3,18:42,Regular,Left,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3982,77.0,14.0,Blocked Shot,NaN,Detroit Red Wings,Home,NaN,3,18:45,Regular,Right,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31
3983,68.0,-29.0,Shot,Wrist Shot,Ottawa Senators,Away,NaN,3,19:48,Regular,Left,NaN,2023-01-01T00:00:00Z,2022020584,2023-01-01 00:00:00+00:00,2022-12-31


In [13]:
sdf.to_csv('Shots_Cleaned.csv')

In [14]:
## Making sure all goals are correct 
number_of_goals_df=sdf[sdf['Event']=="Goal"]
import warnings
with warnings.catch_warnings(record=True):
    number_of_goals_df['Points']=1

import numpy as np
pivot_goals=pd.pivot_table(number_of_goals_df,index=['Game_Id','Team_Name'],values=['Points'],aggfunc=[np.sum])
pd.set_option('display.max_rows', None)
pivot_goals
## Goals match the season results for the red wings for each game
## For game id 2022020203- Red Wings vs Montreal there was a 2-1 shootout score accounting for the extra goals in dataset

sum
                                 Points
Game_Id    Team_Name                   
2022020022 Detroit Red Wings          3
2022020028 Detroit Red Wings          5
           New Jersey Devils          2
2022020044 Detroit Red Wings          4
           Los Angeles Kings          5
2022020072 Chicago Blackhawks         4
           Detroit Red Wings          3
2022020088 Anaheim Ducks              1
           Detroit Red Wings          5
2022020102 Detroit Red Wings          2
           New Jersey Devils          6
2022020111 Boston Bruins              5
           Detroit Red Wings          1
2022020132 Detroit Red Wings          2
           Minnesota Wild             1
2022020144 Buffalo Sabres             8
           Detroit Red Wings          3
2022020164 Detroit Red Wings          3
           Washington Capitals        1
2022020181 Detroit Red Wings          3
2022020190 Detroit Red Wings          3
           New York Rangers           2
2022020203 Detroit Red Wings          3
           Montreal Canadiens         4
2022020217 Detroit Red Wings          2
           New York Rangers           8
2022020237 Detroit Red Wings          3
           Los Angeles Kings          4
2022020255 Anaheim Ducks              3
           Detroit Red Wings          2
2022020272 Detroit Red Wings          7
           San Jose Sharks            4
2022020279 Columbus Blue Jackets      1
           Detroit Red Wings          6
2022020311 Detroit Red Wings          3
2022020325 Arizona Coyotes            3
           Detroit Red Wings          4
2022020348 Detroit Red Wings          2
           Toronto Maple Leafs        4
2022020361 Buffalo Sabres             5
           Detroit Red Wings          4
2022020386 Detroit Red Wings          1
           Vegas Golden Knights       4
2022020394 Columbus Blue Jackets      2
           Detroit Red Wings          4
2022020405 Detroit Red Wings          4
           Tampa Bay Lightning        2
2022020419 Detroit Red Wings          1
           Florida Panthers           5
2022020435 Dallas Stars               3
           Detroit Red Wings          2
2022020460 Carolina Hurricanes        1
2022020467 Detroit Red Wings          1
           Minnesota Wild             4
2022020485 Detroit Red Wings          3
           Ottawa Senators            6
2022020503 Detroit Red Wings          3
           Washington Capitals        4
2022020519 Detroit Red Wings          7
           Tampa Bay Lightning        4
2022020560 Detroit Red Wings          5
           Pittsburgh Penguins        4
2022020563 Buffalo Sabres             6
           Detroit Red Wings          3
2022020584 Detroit Red Wings          4
           Ottawa Senators            2

In [15]:
## Goals all goals in shootout period 
SO=sdf[sdf['Rink_Side']=='SO']
SO_goal=SO[SO['Event']=='Goal']
SO_goal

,X,Y,Event,Shot_Type,Team_Name,Team_Type,Strength,Period,Period_Time,Period_Type,Rink_Side,Empty_Net,Date_Time,Game_Id,DT_Standard,Date
1461,80.0,-1.0,Goal,Snap Shot,Montreal Canadiens,Away,EVEN,5,00:00,Shootout,SO,NaN,2022-11-09T00:00:00Z,2022020203,2022-11-09 00:00:00+00:00,2022-11-08
1463,76.0,-3.0,Goal,Snap Shot,Montreal Canadiens,Away,EVEN,5,00:00,Shootout,SO,NaN,2022-11-09T00:00:00Z,2022020203,2022-11-09 00:00:00+00:00,2022-11-08
1464,-73.0,4.0,Goal,Snap Shot,Detroit Red Wings,Home,EVEN,5,00:00,Shootout,SO,NaN,2022-11-09T00:00:00Z,2022020203,2022-11-09 00:00:00+00:00,2022-11-08
2265,-79.0,-2.0,Goal,Wrist Shot,Detroit Red Wings,Home,EVEN,5,00:00,Shootout,SO,NaN,2022-11-26T01:00:00Z,2022020325,2022-11-26 01:00:00+00:00,2022-11-25
2508,82.0,-5.0,Goal,Wrist Shot,Buffalo Sabres,Away,EVEN,5,00:00,Shootout,SO,NaN,2022-12-01T00:00:00Z,2022020361,2022-12-01 00:00:00+00:00,2022-11-30


In [ ]:
sdf.to_csv('Shots_Cleaned.csv')